In [1]:
from bs4 import BeautifulSoup as bs
import requests
from tqdm.notebook import tqdm
from copy import deepcopy

## парсинг только того, где есть подстрочник (литературный перевод хз норм или нет, не стал брать)

In [41]:
page = open(r'C:\Users\1645286\Downloads\Мансийский язык_ список произведений и авторов - Портал национальных литератур - Россия для всех.html', encoding='utf-8').read()
page = bs(page, 'lxml')

In [42]:
urls = [art.find('a') for art in page.findAll('article')]

In [43]:
import chardet
import httpx

In [44]:
pages = []
for url in tqdm(urls):
    response = requests.get(url['href'])
    #response.encoding = 'utf-8'
    text_page = response.text
    if 'Подстрочник' in text_page:
        cur_article = {}
        cur_article['url'] = url['href']
        cur_article['title'] = url.text
        text_page = bs(response.content, 'html5lib')
        mansi_text = ([p for p in text_page.find('div', {'id': 'tab-source'}).findAll('p')]) # '\n'.join
        rus_text = ([p for p in text_page.find('div', {'id': 'tab-subscript'}).findAll('p')]) #'\n'.join
        cur_article['mansi'] = mansi_text
        cur_article['rus'] = rus_text
        pages.append(deepcopy(cur_article))

  0%|          | 0/40 [00:00<?, ?it/s]

## symbols exploration

In [7]:
def getcwm(x):
    print(x + '̄')

In [8]:
getcwm('Ы')

Ы̄


In [18]:
chars = ["ӯ", "Ӯ", "ӣ", "Ӣ", "ā", "Ā", "ȳ", "Ȳ", "ō", "Ō", "ē", "Ē"]
chars2 = [el + chr(772) for el in "а̄А̄я̄Я̄ӯӮю̄Ю̄о̄О̄е̄Е̄ё̄Ё̄э̄Э̄ӣӢы̄Ы̄".split(chr(772))][:-1]

In [19]:
d = {}
for c in tqdm(chars):
    d[c] = case_data[case_data.target.str.contains(c)].shape[0]
d

  0%|          | 0/12 [00:00<?, ?it/s]

NameError: name 'case_data' is not defined

In [ ]:
d = {}
for c in tqdm(chars2):
    d[c] = case_data[case_data.target.str.contains(c)].shape[0]
d

In [ ]:
chars3 = ['¬', 'Ā', 'ā', 'Ē', 'ē', 'Ō', 'ō', 'ū', '̄', 'Ё', 'ѐ', 'ё', 'є', 'ў', 'ӆ', 'Ӈ', 'ӈ', 'ӊ', 'ӑ', 'ә', 'Ӣ', 'ӣ', 'Ӯ', 'ӯ', '\uf50f', '\uf511', '\uf513', '\uf518', '\uf519', '\uf521', '\uf523', '\uf529', '\uf52d']
chars3

In [ ]:
a = '\uf518'

In [ ]:
case_data[case_data.target.str.contains("ы̄")] 

In [ ]:
print(str(rus_text[3]).replace('<br/>', '\n').replace('<p>', '').replace('</p>', ''))

## pprint

In [9]:
def pprint(d):
    print('Title:', d['title'], end='\n\n')
    print('Mansi:\n', d['mansi'], end='\n\n')
    print('Russian:\n', d['rus'])

In [10]:
pprint(pages[0])

Title: Дай сил подняться мне на немощных крылах...

Mansi:
 [<p>*     *     *</p>, <p><br/>Вāгтāл товлагумтыл нӈх вос лапнувум,<br/>Мā яныттыл вос кāсаланувлум,<br/>Пēс йис мāхманув тох яласлмыт,<br/>Кислквыӈ мāкв тāн сль мувылтлмыт.</p>, <p><br/>Ты хāйтнэ тулыт тāн нумыл ялманыл,<br/>Ты миннэ тулыт ёл-хультуптлманыл,<br/>раныл-мутраныл Трумн туп вāвет,<br/>Нāй-тыр наманыл ат ёрувлавет.</p>, <p><br/>Тāн ньсюм мутраныл хоталь минасыт?<br/>Хт, хотты йисыт сыс тан тыпыгласыт?<br/>Манрыг тыхтал мāн нӈх ат лапēв-а?<br/>Манарын ос хотьютын ёл-пувавēв-а?</p>, <p><br/>Āти, тāн рганыл с сумылласыт,<br/>Тāн пинум лāтӈаныл нӈх лапаясыт,<br/>Мāньси мирув аквтоп товлыӈыг ēмтыс —<br/>Хунь пēс мāхум потрыт хӯнтамлаӈкв вēрмыс.</p>, <p><br/>ква-пыг-пыгрись с нӈх ялтуптавес,<br/>Тāв мутраӈ номтттэ ат тыпылтавес,<br/>Паль сьнэ хтпан тав хӯлуӈкв патаве,<br/>Сам сьнэ хтпан тав ксаллаве.</p>, <p><br/>Товлагум вāгтлыг, нӈх ат лапēгум...<br/>тил мāм хосыт ам сгтым мēгум!<br/>Н

# очистка и выравнивание

In [13]:
import re
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

In [11]:
replace_table = {
    '\uf519': 'о̄',
    '\uf523': 'э̄',
    '\uf50f': 'а̄',
    '\uf511': 'ē',
    '\uf529': 'а̄',
    '\uf518': 'о̄',
    '\uf528': 'Я̄',
    '\uf513': 'ё̄',
    '\uf522': 'Э̄',
    '\uf512': 'Ё̄',
    '\uf50e': 'А̄',
    '\uf521': 'ы̄',
    '\uf52d': 'ю̄',
    'ū': 'ӣ'
}

In [14]:
case_data = pd.read_csv("cleared_v2.csv")
case_data

,target,source
0,Та пыгрисит маим вармаль э̄рнэ поратэт ат вермгыг варункв,Те мальчики не выполнят задание в назначенный срок.
1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёрувлвылув",Мы никогда не забудем этого честного человека.
...,...,...
79928,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄хматын ма̄гыс лю̄льсаӈыг сусхатуӈкве вēрмēгыт.,"Мне кажется, что подобные случаи могут вызывать подозрение."
79929,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
79930,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления меда нет."
79931,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэнк Заппа ёт гастролин минас.","Затем Уэйтс отправился на гастроли с такими музыкантами, как Чарли Рич и Фрэнк Заппа."


In [129]:
html_replace_table = {
    '<p>': '\n',
    '</p>': '',
    '\xa0': ' ',
    '\xad': '',
    '<br/>': '\n',
    '<strong>': '',
    '</strong>': '',
    '<em>': '',
    '</em>': '',
    '\u202f': ''
}

In [147]:
def get_parallel_sentences(rus_list, mansi_list):
    res = []
    for text in deepcopy([rus_list, mansi_list]):
        text = ''.join([str(s) for s in text])
        for old, new in html_replace_table.items():
            text = text.replace(old, new)
        for old, new in replace_table.items():
            text = text.replace(old, new)
        text = text.split('\n')
        to_delete = []
        for i in range(len(text)):
            if not bool(re.findall(r'[а-яА-Я]', text[i])) or 'Подстрочный перевод' in text[i]:
                to_delete.append(text[i])
                continue
            for space in [' ' * j for j in range(10, 1, -1)]:
                text[i] = text[i].replace(space, ' ')
            text[i] = text[i].strip()
        text = [t for t in text if not t in to_delete]
        res.append(text)
    return res

In [146]:
for i, page in enumerate(pages):
    t1, t2 = get_parallel_sentences(page['rus'], page['mansi'])
    if not len(t1) == len(t2):
        print(i)

0
8


[<p><strong>Ам самын патум пвлум</strong></p>,
 <p><br/>Ам самын патум пвлум<br/>Тӯйтыл лап-вōтаставес.<br/>Тыт млты тāл пāвыл пусын хот-свес,<br/>Ам сымум сома лāкква патыс,<br/>Воссыг сымумн ляпа нмāтыр пāвыл āтим,<br/>Туп ам яныгмам пāвлум тот хультыс.<br/>Хосат хот-минасум,<br/>Саль, нврамыг ōлнэ порам тыт ат холас.<br/>Тыхōтал хосат ōлгум,<br/>Туп сымум, тувыл номтум номг пусын.<br/>Ты āӈквал, татык сып,<br/>Хотыл мāн, нврамыт, тлы тахсысув.<br/>Татык тятьтялахтын лāкв,<br/>Āсюм ляпа руматэ хуньт мāнавн вāрыглас...<br/>Саль, воссыг нмхуньт тыйи ат кāсалгум ...<br/>Туп сымумт, тувыл номтумт хультсыт.<br/>Номилум, сома ср ань ты...<br/>Саль, тыт млты тāл уля тс...</p>]

In [148]:
t1, t2 = get_parallel_sentences(pages[7]['rus'], pages[7]['mansi'])

In [149]:
t1, t2

(['Село, в котором я родилась',
  'Село, в котором я родилась',
  'Сегодня заметено белым снегом.',
  'Здесь в прошлом году бушевал пожар,',
  'И сердце моё словно распалось на кусочки.',
  'Ведь роднее моего села нет другого места,',
  'Лишь оно одно живёт в моём сердце.',
  'Я давно покинула село,',
  'Жаль, но моё детство уже прошло.',
  'Сегодня я живу вдали от дома,',
  'Только в сердце и в памяти оно навсегда.',
  'Этот пенёк и тот пригорок,',
  'С которого мы, дети, зимой катались.',
  'То железное колечко качелей,',
  'Которое смастерил нам папин русский друг…',
  'Жаль, больше это не увижу никогда…',
  'Только в сердце и в памяти это осталось.',
  'Помню всё, и живу я прошлым…',
  'Жаль, здесь в прошлом году бушевал пожар…'],
 ['Ам самын патум па̄влум',
  'Ам самын патум па̄влум',
  'Тӯйтыл лап-вōтаставес.',
  'Тыт мо̄лты тāл пāвыл пусын хот-ё̄свес,',
  'Ам сымум сома лāкква патыс,',
  'Воссыг сымумн ляпа нэ̄мāтыр пāвыл āтим,',
  'Туп ам яныгмам пāвлум тот хультыс.',
  'Хосат 

In [144]:
len(t1), len(t2)

(20, 19)

In [117]:
pages[5]['url']

'https://rus4all.ru/mns/20200413/26228/Mame.html'